In [123]:
from langchain_community.utilities import GoogleSearchAPIWrapper
import yfinance
import json
import requests
import os


def get_ticker(inputs):
    company_name = inputs["company_name"]
    print(company_name)
    base_url = "https://www.alphavantage.co/query"
    api_key = os.getenv("ALPAVANTAGE_API_KEY")
    response = requests.get(
        base_url,
        params={
            "function": "SYMBOL_SEARCH",
            "keywords": company_name,
            "apikey": api_key,
        },
    )

    data = response.json()
    try:
        ticker = next(match['1. symbol'] for match in data["bestMatches"] 
                      if match['4. region'] == 'United States')
        return ticker
    except StopIteration:
        return "Ticker Not Found"


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [ ]:
from openai import OpenAI
client = OpenAI()

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4o-mini",
#     tools=functions,
# )

# assistant

In [124]:
get_ticker({"company_name":"Cloudflare"})

Cloudflare


'NET'

In [ ]:
assistant_id = "asst_A3UixVCK7ajFn9ESOBoBScPR"

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)

thread

In [143]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_5XXD2XG3vFXAnq70KV1wOarO', assistant_id='asst_A3UixVCK7ajFn9ESOBoBScPR', cancelled_at=None, completed_at=None, created_at=1740986001, expires_at=1740986601, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_BZ901ANg3cXn1Z4hPK7csKcG', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefini

In [126]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(run_id=run_id, thread_id=thread_id)


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outputs
    )

In [145]:
get_tool_outputs(run.id,thread.id)

Calling function: get_daily_stock_performance with arg {"ticker":"NET"}


[{'output': '"{\\"Open\\":{\\"1733115600000\\":106.0299987793,\\"1733202000000\\":103.3700027466,\\"1733288400000\\":107.0,\\"1733374800000\\":109.4100036621,\\"1733461200000\\":111.2799987793,\\"1733720400000\\":112.9599990845,\\"1733806800000\\":112.8000030518,\\"1733893200000\\":112.3949966431,\\"1733979600000\\":112.4000015259,\\"1734066000000\\":114.8000030518,\\"1734325200000\\":113.9000015259,\\"1734411600000\\":118.2300033569,\\"1734498000000\\":117.5800018311,\\"1734584400000\\":110.3700027466,\\"1734670800000\\":108.0,\\"1734930000000\\":112.1600036621,\\"1735016400000\\":111.75,\\"1735189200000\\":112.1100006104,\\"1735275600000\\":111.9449996948,\\"1735534800000\\":108.6699981689,\\"1735621200000\\":109.0,\\"1735794000000\\":114.5,\\"1735880400000\\":113.4000015259,\\"1736139600000\\":116.3649978638,\\"1736226000000\\":116.6399993896,\\"1736312400000\\":112.4300003052,\\"1736485200000\\":111.1299972534,\\"1736744400000\\":110.0350036621,\\"1736830800000\\":112.4599990845,\\

In [146]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_daily_stock_performance with arg {"ticker":"NET"}


Run(id='run_5XXD2XG3vFXAnq70KV1wOarO', assistant_id='asst_A3UixVCK7ajFn9ESOBoBScPR', cancelled_at=None, completed_at=None, created_at=1740986001, expires_at=1740986601, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1740986005, status='queued', thread_id='thread_BZ901ANg3cXn1Z4hPK7csKcG', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Function

In [149]:
get_run(run.id, thread.id).status

'completed'

In [150]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
user: Please go ahead
assistant: ### Salesforce (CRM) Overview

#### Income Statement Highlights
- **Total Revenue (Latest)**: $34.86 billion (Year over Year improvement)
- **Net Income**: $4.14 billion
- **Diluted EPS (Earnings Per Share)**: $4.20

#### Balance Sheet Highlights
- **Total Assets**: $99.82 billion
- **Total Liabilities**: $40.18 billion
- **Stockholders Equity**: $59.65 billion
- **Net Debt**: $9.54 billion

#### Stock Performance (Last 100 Days)
- **Current Price**: $329.40 (as of the last trading day)
- **Recent High**: $368.58
- **Recent Low**: $292.15

### Analysis

1. **Financial Health**:
   - Salesforce has a solid revenue stream with significant gross profits. The company also demonstrates robust net income, showing profitable growth.

2. **Balance Sheet**:
   - The asset level surpasses liabilities significantly. With total equity of $59.65 billion, Salesforce is relatively strong leverage-wise. However

In [141]:
send_message(thread.id, "Now I want to know if CloudFlare is a good buy.")

Message(id='msg_9k2DuYgohvojtR9NCfgs9gmP', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Now I want to know if CloudFlare is a good buy.'), type='text')], created_at=1740985901, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_BZ901ANg3cXn1Z4hPK7csKcG')